# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [2]:
# import statements
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
import re
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.base import BaseEstimator, TransformerMixin

In [3]:
# load data from database
engine = create_engine('sqlite:///ETL_pipeline_preparation.db')
df = pd.read_sql_table('ETL_preparationTable', engine)
X = df['message']
Y = df[df.columns[4:]]

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens
    

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', RandomForestClassifier())
    ])

In [6]:
def display_results(y_test, y_pred):
    accuracy = (y_pred == y_test).mean()
    print("Accuracy:", accuracy)

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
#Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [8]:
#Train pipeline
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
# predict on test data
y_pred = pipeline.predict(X_test)

# display results
display_results(y_test, y_pred)

Accuracy: related                   0.710640
request                   0.827337
offer                     0.996002
aid_related               0.565344
medical_help              0.922048
medical_products          0.951568
search_and_rescue         0.971095
security                  0.980474
military                  0.967558
water                     0.934348
food                      0.887915
shelter                   0.911439
clothing                  0.985547
money                     0.977860
missing_people            0.990160
refugees                  0.964330
death                     0.951415
other_aid                 0.861316
infrastructure_related    0.928967
transport                 0.954490
buildings                 0.947263
electricity               0.978629
tools                     0.994157
hospitals                 0.988315
shops                     0.996617
aid_centers               0.986777
other_infrastructure      0.950953
weather_related           0.727245
floods    

In [10]:
print(classification_report(y_test, y_pred, target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.78      0.87      0.82      5012
               request       0.47      0.07      0.12      1115
                 offer       0.00      0.00      0.00        26
           aid_related       0.44      0.14      0.22      2726
          medical_help       0.00      0.00      0.00       503
      medical_products       0.00      0.00      0.00       315
     search_and_rescue       0.00      0.00      0.00       188
              security       0.00      0.00      0.00       127
              military       0.00      0.00      0.00       211
                 water       0.00      0.00      0.00       427
                  food       0.58      0.01      0.02       731
               shelter       0.33      0.00      0.01       574
              clothing       0.00      0.00      0.00        93
                 money       0.00      0.00      0.00       142
        missing_people       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
parameters = {
        'clf__n_estimators': [50, 100, 200],
        'clf__min_samples_split': [2, 3, 4]
    }

cv = GridSearchCV(pipeline, param_grid=parameters, cv=5,scoring='f1_micro', n_jobs=-1)

In [11]:
parameters = {
        'clf__n_estimators': [10, 50, 100],
        'clf__min_samples_split': [2, 3, 4]
    }

cv = GridSearchCV(pipeline, param_grid=parameters, cv=5,scoring='f1_micro', n_jobs=-1)

In [ ]:
cv.fit(X_train, y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [17]:
cv.fit(X_train, y_train)

# predict on test data
y_pred_final = cv.predict(X_test)

# display results
display_results(y_test, y_pred_final)

KeyboardInterrupt: 

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [12]:
# Build a custom transformer which will extract the starting verb of a sentence
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    """
    Starting Verb Extractor class
    
    This class extract the starting verb of a sentence,
    creating a new feature for the ML classifier
    """

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    # Given it is a tranformer we can return the self 
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [13]:
pipeline2 = Pipeline([
        ('features', FeatureUnion([
            
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),
    
        ('clf', RandomForestClassifier())
    ])

In [14]:
pipeline2.fit(X_train, y_train)

# predict on test data
y_pred = pipeline2.predict(X_test)

# display results
display_results(y_test, y_pred)

print(classification_report(y_test, y_pred, target_names=Y.columns.values))

Accuracy: related                   0.726015
request                   0.824569
offer                     0.996002
aid_related               0.567958
medical_help              0.921125
medical_products          0.950800
search_and_rescue         0.970787
security                  0.980320
military                  0.967558
water                     0.934041
food                      0.887300
shelter                   0.910670
clothing                  0.985394
money                     0.978014
missing_people            0.990006
refugees                  0.964176
death                     0.951415
other_aid                 0.860547
infrastructure_related    0.928659
transport                 0.954336
buildings                 0.947263
electricity               0.978629
tools                     0.994157
hospitals                 0.988161
shops                     0.996617
aid_centers               0.986777
other_infrastructure      0.950953
weather_related           0.727399
floods    

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [16]:
import pickle

pickle.dump(cv, open('MLclassifier.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.